In [ ]:
from pathlib import Path
from warnings import filterwarnings

import iris
import matplotlib.pyplot as plt
import numpy as np
from wildfires.analysis import cube_plotting
from wildfires.data import homogenise_time_coordinate

from jules_output_analysis.utils import cube_1d_to_2d

filterwarnings("ignore", ".*divide by zero.*")

In [ ]:
source_dir = Path("/work/scratch-nopw/alexkr/multi_spinup4/jules_output")
assert source_dir.is_dir()

In [ ]:
cubes_2d = []
targets = ("decomposable", "resistant")
for target in targets:
    cube = homogenise_time_coordinate(
        iris.load(
            str(source_dir / "*SPINUP5*Monthly*.nc"),
            constraints=f"C in {target} plant material, gridbox total",
        )
    ).concatenate_cube()

    cube_2d = cube_1d_to_2d(cube)
    cubes_2d.append(cube_2d)

    avg_cube_2d = cube_2d.collapsed("time", iris.analysis.MEAN)
    max_cube_2d = cube_2d.collapsed("time", iris.analysis.MAX)

    fig = cube_plotting(avg_cube_2d, title=f"{target} mean")

    fig = cube_plotting(max_cube_2d, title=f"{target} max")

    std_cube_2d = avg_cube_2d.copy(data=np.std(cube_2d.data, axis=0))
    std_cube_2d

    fig = cube_plotting(std_cube_2d, title=f"{target} std")

    fig = cube_plotting(std_cube_2d / avg_cube_2d, title=f"{target} std / mean")

In [ ]:
ratio = cubes_2d[1] / cubes_2d[0]

In [ ]:
mean_ratio = ratio.collapsed("time", iris.analysis.MEAN)

In [ ]:
data = mean_ratio.data.ravel()
data = data[data < 200]
_ = plt.hist(data, bins=200)

In [ ]:
fig = cube_plotting(
    mean_ratio,
    title=f"{targets[1]} / {targets[0]}",
    vmin=0,
    vmax=150,
    fig=plt.figure(figsize=(7, 3.2), dpi=200),
)